In [16]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 50.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [17]:
!pip install tf-keras


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

PRODUCTS_CSV = "../data/productos.csv"
CLIENTES_CSV = "../data/clientes.csv"
HISTORIAL_COMPRAS = "../data/historial_compras.csv"

productos = pd.read_csv(PRODUCTS_CSV)
clientes = pd.read_csv(CLIENTES_CSV)
historial_compras = pd.read_csv(HISTORIAL_COMPRAS)

In [7]:
print("Productos:", productos.shape)
print("Clientes:", clientes.shape)
print("historial_compras:", historial_compras.shape)

print("Columnas productos:", productos.columns.tolist())
print("Columnas clientes:", clientes.columns.tolist())
print("Columnas historial compras:", historial_compras.columns.tolist())

Productos: (10, 6)
Clientes: (25, 6)
historial_compras: (220, 5)
Columnas productos: ['id_producto', 'nombre_producto', 'categoria', 'subcategoria', 'precio', 'descripcion']
Columnas clientes: ['id_cliente', 'nombre', 'email', 'ciudad', 'pais', 'fecha_registro']
Columnas historial compras: ['id_compra', 'id_cliente', 'id_producto', 'cantidad', 'fecha_compra']


In [8]:
# Merge compras con products
compras_prod = historial_compras.merge(productos, on="id_producto", how="left")
print("Dataset compras por producto unido:", compras_prod.shape)
compras_prod.head()

Dataset compras por producto unido: (220, 10)


,id_compra,id_cliente,id_producto,cantidad,fecha_compra,nombre_producto,categoria,subcategoria,precio,descripcion
0,1,1,2,1,2024-05-20,Smartphone Nexus 5G,Electrónica,Smartphones,899.50,El Smartphone Nexus 5G redefine la experiencia...
1,2,2,4,2,2024-05-21,Camiseta Deportiva Ultralight,Ropa,Camisetas,35.00,Camiseta técnica de alto rendimiento para depo...
2,3,3,1,1,2024-05-22,Laptop Gamer Pro,Electrónica,Portátiles,1899.99,Laptop de alto rendimiento para gaming y creac...
3,4,4,5,1,2024-05-23,Zapatillas Urbanas Fit,Ropa,Calzado,85.75,Calzado deportivo y casual diseñado para el co...
4,5,5,3,1,2024-05-24,Auriculares Inalámbricos X,Electrónica,Audio,125.00,Auriculares Bluetooth de diadema diseñados par...


In [9]:
# Merge con clients
compras_full = compras_prod.merge(clientes, on="id_cliente", how="left")
print("Dataset compras full:", compras_full.shape)
compras_full.head()

Dataset compras full: (220, 15)


,id_compra,id_cliente,id_producto,cantidad,fecha_compra,nombre_producto,categoria,subcategoria,precio,descripcion,nombre,email,ciudad,pais,fecha_registro
0,1,1,2,1,2024-05-20,Smartphone Nexus 5G,Electrónica,Smartphones,899.50,El Smartphone Nexus 5G redefine la experiencia...,Ana Martínez,ana.m@email.com,Madrid,España,2024-05-10
1,2,2,4,2,2024-05-21,Camiseta Deportiva Ultralight,Ropa,Camisetas,35.00,Camiseta técnica de alto rendimiento para depo...,Carlos Sánchez,carlos.s@email.com,Ciudad de México,México,2024-05-11
2,3,3,1,1,2024-05-22,Laptop Gamer Pro,Electrónica,Portátiles,1899.99,Laptop de alto rendimiento para gaming y creac...,María López,maria.l@email.com,Bogotá,Colombia,2024-05-12
3,4,4,5,1,2024-05-23,Zapatillas Urbanas Fit,Ropa,Calzado,85.75,Calzado deportivo y casual diseñado para el co...,Jorge García,jorge.g@email.com,Buenos Aires,Argentina,2024-05-13
4,5,5,3,1,2024-05-24,Auriculares Inalámbricos X,Electrónica,Audio,125.00,Auriculares Bluetooth de diadema diseñados par...,Valentina Pérez,valentina.p@email.com,Santiago,Chile,2024-05-14


In [14]:
client_profiles = (
    compras_full.groupby("id_cliente")
    .apply(lambda g: " ".join(g["nombre_producto"].tolist() + g["descripcion"].tolist()))
    .reset_index(name="texto_cliente")
)

# Agregar información de clientes (ciudad, país)
client_profiles = client_profiles.merge(clientes, on="id_cliente")
client_profiles["texto_cliente"] = (
    client_profiles["nombre"] + " " +
    client_profiles["ciudad"] + " " +
    client_profiles["pais"] + " " +
    client_profiles["texto_cliente"]
)

print("Perfiles de clientes generados:", client_profiles.shape)
client_profiles.head()

Perfiles de clientes generados: (20, 7)


/var/folders/sl/2x216nz107xcfzxlmr7tr47m0000gn/T/ipykernel_60686/1070130770.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: " ".join(g["nombre_producto"].tolist() + g["descripcion"].tolist()))


,id_cliente,texto_cliente,nombre,email,ciudad,pais,fecha_registro
0,1,Ana Martínez Madrid España Smartphone Nexus 5G...,Ana Martínez,ana.m@email.com,Madrid,España,2024-05-10
1,2,Carlos Sánchez Ciudad de México México Camiset...,Carlos Sánchez,carlos.s@email.com,Ciudad de México,México,2024-05-11
2,3,María López Bogotá Colombia Laptop Gamer Pro Z...,María López,maria.l@email.com,Bogotá,Colombia,2024-05-12
3,4,Jorge García Buenos Aires Argentina Zapatillas...,Jorge García,jorge.g@email.com,Buenos Aires,Argentina,2024-05-13
4,5,Valentina Pérez Santiago Chile Auriculares Ina...,Valentina Pérez,valentina.p@email.com,Santiago,Chile,2024-05-14


In [19]:
# Remover Stopwords y Lematizar en español en texto_cliente
import spacy
nlp = spacy.load("es_core_news_sm")
print("spaCy cargado: usando lematización y stopwords de spaCy.")

def preprocess_spacy(text):
    doc = nlp(str(text).lower())
    tokens = [t.lemma_ for t in doc if not t.is_stop and t.is_alpha]
    return " ".join(tokens)

# Procesar texto de clientes
client_profiles["texto_cliente_proc"] = client_profiles["texto_cliente"].apply(preprocess_spacy)

# Revisa resultados
client_profiles[["id_cliente", "nombre", "texto_cliente", "texto_cliente_proc"]].head()

spaCy cargado: usando lematización y stopwords de spaCy.


,id_cliente,nombre,texto_cliente,texto_cliente_proc
0,1,Ana Martínez,Ana Martínez Madrid España Smartphone Nexus 5G...,ana martínez madrid españa smartphonir nexus g...
1,2,Carlos Sánchez,Carlos Sánchez Ciudad de México México Camiset...,carlos sánchez ciudad méxico méxico camiseta d...
2,3,María López,María López Bogotá Colombia Laptop Gamer Pro Z...,maría lópez bogotá colombia laptop gamer pro z...
3,4,Jorge García,Jorge García Buenos Aires Argentina Zapatillas...,jorge garcía aires argentina zapatilla urbano ...
4,5,Valentina Pérez,Valentina Pérez Santiago Chile Auriculares Ina...,valentina pérez santiago chile auricular inalá...


In [20]:
from sentence_transformers import SentenceTransformer

print("SentenceTransformers disponible. Usando modelo multilingüe.")
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
st_model = SentenceTransformer(model_name)

/Users/erichuiza/Documents/pucp/miería web/dev/sesion-de-clase-06/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceTransformers disponible. Usando modelo multilingüe.


In [23]:
import random

# Productos únicos con texto procesado
productos_unicos = productos[["id_producto", "nombre_producto", "descripcion"]].drop_duplicates(subset="id_producto").reset_index(drop=True)

# Preprocesar productos con spaCy
productos_unicos["nombre_producto_proc"] = productos_unicos["nombre_producto"].apply(preprocess_spacy)
productos_unicos["descripcion_proc"] = productos_unicos["descripcion"].apply(preprocess_spacy)

# Listas de textos
nombres_textos = productos_unicos["nombre_producto_proc"].tolist()
productos_textos = productos_unicos["descripcion_proc"].tolist()
clientes_textos = client_profiles["texto_cliente_proc"].tolist()

# Todos los textos para embeddings
todos_textos = nombres_textos + productos_textos + clientes_textos

# Mostrar 5 elementos aleatorios
muestra_aleatoria = random.sample(todos_textos, 5)

print("Muestra aleatoria de la lista 'todos_textos':")
for i, texto in enumerate(muestra_aleatoria):
    print(f"{i+1}. {texto}")

Muestra aleatoria de la lista 'todos_textos':
1. calzado deportivo casual diseñado confort diario superior malla transpirable suela espuma memoria adaptar forma pie proporcionar soporte inigualable amortiguación superior suela goma duradero ofrecer excelente tracción diverso superficie diseño minimalista elegante ideal combinar atuendo jeans pantalón deporte disponible talla unisex
2. luis hernández lima perú libro círculo mágico smartphonir nexus g auricular inalámbrico x camiseta deportivo ultralight zapatilla urbano fit laptop gamer pro laptop gamer pro laptop gamer pro laptop gamer pro laptop gamer pro smartphonir nexus g novela fantasía épico sumergar reino eterea historia elara joven magar dominar ancestral salvar pueblo inminente autoro tejar narrativa intrincado lleno personaje complejo criatura mitológica giro inesperado mundo construido riqueza detalle hacer sentir perfecto fan saga fantasía tema heroísmo traición redención smartphonir nexus g redefinar experiencia móvil pant

In [46]:
# Construir textos para ambos grupos
clientes_textos = client_profiles["texto_cliente"].tolist()
productos_textos = productos_unicos["nombre_producto"].tolist()

# Concatenar para un solo encode
todos_textos = clientes_textos + productos_textos

# Generar embeddings de todo junto
embeddings = st_model.encode(todos_textos, convert_to_numpy=True, show_progress_bar=True)

# Separar los embeddings en clientes y productos
n_clientes = len(clientes_textos)
n_productos = len(productos_textos)

client_embeddings = embeddings[:n_clientes]
prod_embeddings = embeddings[n_clientes:]

Batches: 100%|██████████| 1/1 [00:00<00:00, 11.46it/s]


In [47]:
out_dir = "logs/embeddings"
os.makedirs(out_dir, exist_ok=True)

def make_label(text, n_words=5):
    return " ".join(text.split()[:n_words])

# Concatenar embeddings (clientes + productos)
all_embeddings = np.vstack([client_embeddings, prod_embeddings])

# Crear metadata
metadata = pd.DataFrame({
    "label": (
        [make_label(t, 5) for t in client_profiles["texto_cliente"].tolist()] +
        [make_label(t, 5) for t in productos_unicos["nombre_producto"].tolist()]
    ),
    "tipo": (
        ["cliente"] * len(client_profiles) +
        ["producto"] * len(productos_unicos)
    ),
    "texto_completo": (
        client_profiles["texto_cliente"].tolist() +
        productos_unicos["nombre_producto"].tolist()
    )
})

# Guardar metadata como TSV
metadata_path = os.path.join(out_dir, "metadata.tsv")
metadata.to_csv(metadata_path, sep="\t", index=False)

print("Metadata guardada en:", metadata_path)

# Guardar embeddings para TensorBoard
emb_var = tf.Variable(all_embeddings, name="embeddings")
ckpt = tf.train.Checkpoint(embedding=emb_var)
ckpt.save(os.path.join(out_dir, "embedding.ckpt"))

print("Embeddings guardados en:", out_dir)
print("Ejecuta en terminal: tensorboard --logdir logs/embeddings")

Metadata guardada en: logs/embeddings/metadata.tsv
Embeddings guardados en: logs/embeddings
Ejecuta en terminal: tensorboard --logdir logs/embeddings


In [48]:
# Normalizar embeddings de clientes y productos (una vez)
client_embeddings_norm = client_embeddings / np.linalg.norm(client_embeddings, axis=1, keepdims=True)
prod_embeddings_norm = prod_embeddings / np.linalg.norm(prod_embeddings, axis=1, keepdims=True)


def top_k_products_for_client(client_idx, k=3):
    """
    Dado un cliente, devuelve los k productos más similares
    que aún NO ha comprado.
    """
    q = client_embeddings_norm[client_idx]  # (dim,)
    sims = np.dot(prod_embeddings_norm, q)  # (num_productos,)

    # Productos ya comprados por este cliente
    cliente_id = client_profiles.iloc[client_idx]["id_cliente"]
    comprados = set(compras_full.loc[compras_full["id_cliente"] == cliente_id, "id_producto"].tolist())

    # Ordenar por similitud (descendente)
    topk_idx = sims.argsort()[::-1]

    # Filtrar productos que ya compró
    recomendados = []
    for idx in topk_idx:
        pid = productos_unicos.iloc[idx]["id_producto"]
        if pid not in comprados:
            recomendados.append((
                idx,
                sims[idx],
                pid,
                productos_unicos.iloc[idx]["nombre_producto"],
                productos_unicos.iloc[idx]["descripcion"]
            ))
        if len(recomendados) >= k:
            break
    return recomendados


# Ejemplo: recomendar productos al cliente 10 que no haya comprado
cliente_id = client_profiles.iloc[19]["id_cliente"]
print(f"Cliente {cliente_id}: {client_profiles.iloc[19]['texto_cliente'][:120]}...")
print("Top-3 productos recomendados (no comprados):")
for idx, score, pid, nombre, desc in top_k_products_for_client(19, k=3):
    print(f" - id_producto={pid}, score={score:.4f}, nombre: {nombre}, desc: {desc[:80]}...")


Cliente 20: Mariana León Encarnación Paraguay Cámara de Acción 4K Laptop Gamer Pro Laptop Gamer Pro Cámara de Acción 4K Auriculares ...
Top-3 productos recomendados (no comprados):
 - id_producto=5, score=0.3286, nombre: Zapatillas Urbanas Fit, desc: Calzado deportivo y casual diseñado para el confort diario. La parte superior de...
 - id_producto=2, score=0.2881, nombre: Smartphone Nexus 5G, desc: El Smartphone Nexus 5G redefine la experiencia móvil. Su pantalla OLED de 6.7 pu...
 - id_producto=7, score=0.2723, nombre: Set de Ollas Premium, desc: Juego de 5 ollas de acero inoxidable de alta calidad. Cada olla cuenta con una b...


In [ ]:
# tensorboard --logdir logs/embeddings